# Processing PDFs using Data Prep Kit

This notebook will introduce DPK and showcase some of it's capabilities.

Here is the workflow:

- pdf2parquet: Extract text from PDF documents
- docid: compute hashes
- exact dedupe : filter out identical documents
- fuzzy dedupe : filter out 'near duplicates'
- document quality: scoring documents for quality

![](https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/intro/images/data-prep-kit-3-workflow.png)


## How to run this notebook

Two options:

- **Option 1 - Google Colab:** easiest option.  no setup required.  Click this link to open this on google colab.  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/dpk_intro_1_python.ipynb)
- **Option 2 - Local python dev environment:**  Setup using this [guide](../../../README.md#-getting-started)

The notebook will work as in both environments

## Step-1: Figure out Runtime Environment

### 1.1 - Determine runtime

Determine if we are running on Google colab or local python environment

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


### 1.2 - Install dependencies if running on Google Colab

In [2]:
if RUNNING_IN_COLAB:
    ! pip install  --default-timeout=100  \
        data-prep-toolkit-transforms[ray,all]==1.0.0a4

### 1.3 - Restart Runtime

After installing dependencies, be sure <font color="red">restart runtime</font>, so libraries will be loaded

You do this by going to **`Runtime --> Restart Session`**

Then you can continue to the next step (no need to re-run the notebook)

## Step-2: Configuration  & Utils

### 2.1 - Basic Config

In [3]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


In [4]:
import os

## Configuration
class MyConfig:
    pass

MY_CONFIG = MyConfig ()

MY_CONFIG.INPUT_DATA_DIR = 'input'
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

### 2.2 - Setup input/outpur directories

In [5]:
import os, sys
import shutil

shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

output_text_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_text_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_fuzzy_dedupe_out')
output_doc_quality_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_doc_quality_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)
print ("✅ Cleared output directory")

✅ Cleared output directory


### 2.3 - Handy Utils

In [6]:
import os
import requests
from humanfriendly import format_size
import pandas as pd
import glob

## Reads parquet files in a folder into a pandas dataframe
def read_parquet_files_as_df (parquet_dir):
    parquet_files = glob.glob(f'{parquet_dir}/*.parquet')
    # read each parquet file into a DataFrame and store in a list
    dfs = [pd.read_parquet (f) for f in parquet_files]
    dfs = [df for df in dfs if not df.empty]  # filter out empty dataframes
    # Concatenate all DataFrames into a single DataFrame
    if len(dfs) > 0:
        data_df = pd.concat(dfs, ignore_index=True)
        return data_df
    else:
        return pd.DataFrame() # return empty df
# ------------

## Step-3: Inspect the Data

We will use simple PDFs.  The files are [here](https://github.com/IBM/data-prep-kit/tree/dev/examples/notebooks/pdf-processing-1/input/)

- [earth.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/earth.pdf) and exact duplicate [earth-copy.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/earth-copy.pdf)
- [earth2.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/earth2.pdf) almost similar to earth.pdf (ONE word difference!)
- [mars.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/mars.pdf)
- [spam.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/spam.pdf) - contains spammy contents
- [lorem.pdf](https://github.com/IBM/data-prep-kit/blob/dev/examples/notebooks/pdf-processing-1/input/lorem.pdf) - contains 'lorem ipsum' placeholder


### 3.1 -Download Data if running on Google Colab

In [7]:
if RUNNING_IN_COLAB:
    !mkdir -p '{MY_CONFIG.INPUT_DATA_DIR}'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/earth.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing-1/input/earth.pdf'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/earth-copy.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing-1/input/earth-copy.pdf'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/earth2.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing-1/input/earth2.pdf'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/mars.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing/input/mars.pdf'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/spam.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing/input/spam.pdf'
    !wget -O '{MY_CONFIG.INPUT_DATA_DIR}/lorem.pdf'  'https://raw.githubusercontent.com/IBM/data-prep-kit/dev/examples/notebooks/pdf-processing/input/lorem.pdf'

## Step-4: Extract Data from PDF (pdf2parquet)

This step we will read PDF files and extract the text data.

[Pdf2Parquet documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/pdf2parquet/README.md)

We use the [Docling package](https://github.com/DS4SD/docling).


### 4.1 - Execute

In [8]:
%%time 

from dpk_pdf2parquet.transform_python import Pdf2Parquet
from dpk_pdf2parquet.transform import pdf2parquet_contents_types

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_text_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
                    output_folder= output_text_dir,
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,   # markdown
                    #    pdf2parquet_contents_type=pdf2parquet_contents_types.JSON   # JSON
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_text_out'



00:15:20 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
00:15:20 INFO - pipeline id pipeline_id
00:15:20 INFO - code location None
00:15:20 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_text_out
00:15:20 INFO - data factory data_ max_files -1, n_sample -1
00:15:20 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
00:15:20 INFO - orchestrator pdf2parquet started at 2025-01-21 00:15:20
00:15:20 INFO - Number of files is 6, source profile {'max_file_size': 0.055823326110839844, 'min_file_size': 0.0237159729

✅ Stage:1 completed successfully
CPU times: user 21 s, sys: 1.65 s, total: 22.7 s
Wall time: 10.3 s


### 4.2 - Inspect Generated output

Here we should see one entry per input file processed.

In [9]:
print ("Displaying contents of : ", output_text_dir)
output_df = read_parquet_files_as_df(output_text_dir)
# print ("Output dimensions (rows x columns)= ", output_df.shape)
output_df.head()

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Displaying contents of :  output/01_text_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,spam.pdf,Free xxx,1,0,2,5a9d562d-ba87-4b2f-954d-fd9e7aece509,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-21T00:15:28.419184,0.660825,spam.pdf
1,earth2.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,6fdf34dd-3e36-4311-9d85-eaa2c22146dd,10729312978404042321,pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,610,2025-01-21T00:15:26.512279,0.691329,earth2.pdf
2,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-21T00:15:27.756719,0.615258,mars.pdf
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-21T00:15:24.915376,1.019706,earth-copy.pdf
4,lorem.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,8a86db6e-0ac6-480c-9a86-d4ac383e2589,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-21T00:15:27.139779,0.625818,lorem.pdf



### 4.3 - Understand the output

Here are some interesting attributes to note:

- **filename** : original filename
- **contents** : text
- **document_id**: unique id (UUID) assignd to this document
- **document_hash**: hash of documents
- **hash** : hash of `contents` column
- **pdf_convert_time** : time to convert this pdf in seconds

**Note: you should notice the hash values are identical for the duplicate documents**

Let's inspect the **contents** column.

In [10]:
print (output_df.iloc[0, ]['contents'])

Free xxx


In [11]:
print (output_df.iloc[1, ]['contents'])


## Earth

## Solar System

Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.

For more details about the Solar system see Chapter 1.

## Earth

Earth is the third planet from the Sun. It's our home planet. Earth is the only place we know of with life.

Basic facts about Earth:

- · Distance from the Sun: Average of 149.6 million kilometers (93 million miles)
- · Moons: One moon, called Luna or simply "the Moon".
- · Rotation Period: 24 hours (one day)


## Step-5:  Create DOC ID for Documents

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set **hash_column** to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set **int_id_column** to the name of the column, where you want to store it.

**This step is a pre-requisite for fuzzy dedup** in the pipeline.

[DocID documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/doc_id)

### 5.1 - Execute

In [12]:
%%time 

from dpk_doc_id.transform_python import DocID

STAGE = 2
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_text_dir}' --> output='{output_docid_dir}'\n", flush=True)

result = DocID(input_folder= output_text_dir,
        output_folder= output_docid_dir,
        doc_id_doc_column= "contents",
        doc_id_hash_column= "doc_hash",
        # doc_id_int_column= "doc_id",
        doc_id_int_column= "int_id_column",
        #doc_id_start_id= 5
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")


🏃🏼 STAGE-2: Processing input='output/01_text_out' --> output='output/02_docid_out'



00:15:28 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'doc_hash', 'int_column': 'int_id_column', 'start_id': 0}
00:15:28 INFO - pipeline id pipeline_id
00:15:28 INFO - code location None
00:15:28 INFO - data factory data_ is using local data access: input_folder - output/01_text_out output_folder - output/02_docid_out
00:15:28 INFO - data factory data_ max_files -1, n_sample -1
00:15:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:15:28 INFO - orchestrator doc_id started at 2025-01-21 00:15:28
00:15:28 INFO - Number of files is 6, source profile {'max_file_size': 0.010061264038085938, 'min_file_size': 0.0055408477783203125, 'total_file_size': 0.049612998962402344}
00:15:28 INFO - Completed 1 files (16.67%) in 0.0 min
00:15:28 INFO - Completed 2 files (33.33%) in 0.0 min
00:15:28 INFO - Completed 3 files (50.0%) in 0.0 min
00:15:28 INFO - Com

✅ Stage:2 completed successfully
CPU times: user 20.5 ms, sys: 2.75 ms, total: 23.2 ms
Wall time: 18.6 ms


### 5.2 - Inspect Generated output

You would see a new columns **hash** and **docid** 

In [13]:
print ("Displaying contents of : ", output_docid_dir)
output_df = read_parquet_files_as_df(output_docid_dir)
output_df.head()

Displaying contents of :  output/02_docid_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column
0,spam.pdf,Free xxx,1,0,2,5a9d562d-ba87-4b2f-954d-fd9e7aece509,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-21T00:15:28.419184,0.660825,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5
1,earth2.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,6fdf34dd-3e36-4311-9d85-eaa2c22146dd,10729312978404042321,pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,610,2025-01-21T00:15:26.512279,0.691329,earth2.pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,2
2,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-21T00:15:27.756719,0.615258,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-21T00:15:24.915376,1.019706,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0
4,lorem.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,8a86db6e-0ac6-480c-9a86-d4ac383e2589,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-21T00:15:27.139779,0.625818,lorem.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3


## Step-6: Eliminate Duplicate Documents

We have 2 exact duplicates: **earth.pdf** , **earth-copy.pdf**

Note how **doc_hash** for these documents are the same.

[Exact dedupe information](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/ededup)

### 6.1 - Execute

In [14]:
%%time 

from dpk_ededup.transform_python import Ededup

STAGE = 3
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_docid_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_docid_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="doc_hash"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_docid_out' --> output='output/03_exact_dedupe_out'



00:15:28 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'doc_hash', 'use_snapshot': False, 'snapshot_directory': None}
00:15:28 INFO - pipeline id pipeline_id
00:15:28 INFO - code location None
00:15:28 INFO - data factory data_ is using local data access: input_folder - output/02_docid_out output_folder - output/03_exact_dedupe_out
00:15:28 INFO - data factory data_ max_files -1, n_sample -1
00:15:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:15:28 INFO - orchestrator ededup started at 2025-01-21 00:15:28
00:15:28 INFO - Number of files is 6, source profile {'max_file_size': 0.01116180419921875, 'min_file_size': 0.006641387939453125, 'total_file_size': 0.05621051788330078}
00:15:28 INFO - Starting from the beginning
00:15:28 INFO - Completed 1 files (16.67%) in 0.0 min
00:15:28 INFO - Completed 2 files (33.33%) in 0.0 min
00:15:28 INFO - C

✅ Stage:3 completed successfully
CPU times: user 25.1 ms, sys: 2.26 ms, total: 27.4 ms
Wall time: 22 ms


### 6.2 - Inspect Generated output

You can see one of **earth.pdf** or **earth-copy.pdf** will be eliminated.

In [15]:
input_df = read_parquet_files_as_df(output_docid_dir)
output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

print ("Displaying contents of : ", output_exact_dedupe_dir)
output_df.head()

Input files before exact dedupe : 6
Output files after exact dedupe : 5
Duplicate files removed :   1
Displaying contents of :  output/03_exact_dedupe_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column,removed
0,spam.pdf,Free xxx,1,0,2,5a9d562d-ba87-4b2f-954d-fd9e7aece509,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-21T00:15:28.419184,0.660825,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5,[]
1,earth2.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,6fdf34dd-3e36-4311-9d85-eaa2c22146dd,10729312978404042321,pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,610,2025-01-21T00:15:26.512279,0.691329,earth2.pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,2,[]
2,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-21T00:15:27.756719,0.615258,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4,[]
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-21T00:15:24.915376,1.019706,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0,[]
4,lorem.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,8a86db6e-0ac6-480c-9a86-d4ac383e2589,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-21T00:15:27.139779,0.625818,lorem.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3,[]


## Step-7: Fuzzy Dedupe

In previous step, we removed **exact duplicates (identical documents)**.

Fuzzy de-dupe can further filter out documents that are **not exactly identical, but nearly identical**

For example imagine two documents with one extra blank line.  For our purposes they are the same.

[Fuzzy dedupe documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/fdedup)

### Tweaking the threshold

**`jaccard_similarity_threshold`** is the parameter used to tweak similarities between documents.  It's value is between 0 and 1.0.  Values close to 1.0 means more strict checking (fewer documents will qualify).  Lower threshold means more leniant matches (more documents will qualify)

Adjust this value to find what works for your documents

### 7.1 - Execute

In [16]:
%%time 

from dpk_fdedup.transform_python import Fdedup

STAGE = 4
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_fuzzy_dedupe_dir}'\n", flush=True)

result = Fdedup(input_folder=output_exact_dedupe_dir,
                output_folder=output_fuzzy_dedupe_dir,
                contents_column= "contents",
                # document_id_column= "doc_id",
                document_id_column= "int_id_column",
                num_permutations= 112,
                num_bands= 14,
                num_minhashes_per_band= 8,
                jaccard_similarity_threshold = 0.8, # between 0 - 1.  higher means more strict checking
                operation_mode="filter_duplicates",
                # operation_mode="annotate",
                ).transform()
# if result == 0:
#     print (f"✅ Stage:{STAGE} completed successfully")
# else:
#     raise Exception (f"❌ Stage:{STAGE}  failed (result={result})")

🏃🏼 STAGE-4: Processing input='output/03_exact_dedupe_out' --> output='output/04_fuzzy_dedupe_out'



00:15:28 INFO - Starting SignatureCalculation step
00:15:28 INFO - Got parameters for SignatureCalculation
00:15:28 INFO - minhash parameters are : {'document_id_column': 'int_id_column', 'contents_column': 'contents', 'seed': 42, 'num_permutations': 112, 'jaccard_similarity_threshold': 0.8, 'word_shingle_size': 5, 'num_bands': 14, 'num_minhashes_per_band': 8, 'num_segments': 1, 'shingle_option': 'word'}
00:15:28 INFO - data factory scdata_ is using local configuration without input/output path
00:15:28 INFO - data factory scdata_ max_files -1, n_sample -1
00:15:28 INFO - data factory scdata_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:15:28 INFO - pipeline id pipeline_id
00:15:28 INFO - code location None
00:15:28 INFO - data factory data_ is using local data access: input_folder - output/03_exact_dedupe_out output_folder - output/04_fuzzy_dedupe_out
00:15:28 INFO - data factory data_ max_fil

CPU times: user 301 ms, sys: 71.6 ms, total: 373 ms
Wall time: 265 ms


### 7.2 - Inspect Output

FuzzyDedupe will write documents that are filtered in **output/04_fuzzy_dedupe_out/cleaned** folder

You will notice only one **earth.pdf** made it!  So fuzzy dedupe did filter out the almost identical doc.

In [17]:
input_df = read_parquet_files_as_df(output_exact_dedupe_dir)
output_df = read_parquet_files_as_df(os.path.join(output_fuzzy_dedupe_dir, "cleaned"))

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Near duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

print ("Displaying contents of : ", output_fuzzy_dedupe_dir)
output_df.head()

Input files before exact dedupe : 5
Output files after exact dedupe : 4
Near duplicate files removed :   1
Displaying contents of :  output/04_fuzzy_dedupe_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column,removed
0,spam.pdf,Free xxx,1,0,2,5a9d562d-ba87-4b2f-954d-fd9e7aece509,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-21T00:15:28.419184,0.660825,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5,[]
1,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-21T00:15:27.756719,0.615258,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4,[]
2,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-21T00:15:24.915376,1.019706,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0,[]
3,lorem.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,8a86db6e-0ac6-480c-9a86-d4ac383e2589,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-21T00:15:27.139779,0.625818,lorem.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3,[]


## Step-8: Document Quality

This handy plugin will score documents across many metrics.

Here we will look for 'bad words' metric.

[Document quality documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/doc_quality)

### 8.1 - Execute

In [18]:
%%time 

from dpk_doc_quality.transform_python import DocQuality

STAGE = 5
output_fuzzy_dedupe_cleaned_dir = os.path.join(output_fuzzy_dedupe_dir, "cleaned")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_fuzzy_dedupe_cleaned_dir}' --> output='{output_doc_quality_dir}'\n", flush=True)

result = DocQuality(input_folder=output_fuzzy_dedupe_cleaned_dir,
                    output_folder= output_doc_quality_dir,
                    docq_text_lang = "en",
                    docq_doc_content_column ="contents",
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed (result={result})")

🏃🏼 STAGE-5: Processing input='output/04_fuzzy_dedupe_out/cleaned' --> output='output/05_doc_quality_out'



00:15:28 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/home/sujee/apps/anaconda3/envs/dpk-2-pdf-processing/lib/python3.11/site-packages/dpk_doc_quality/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x76b11c9c4990>}
00:15:28 INFO - data factory docq_ is using local configuration without input/output path
00:15:28 INFO - data factory docq_ max_files -1, n_sample -1
00:15:28 INFO - data factory docq_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:15:28 INFO - pipeline id pipeline_id
00:15:28 INFO - code location None
00:15:28 INFO - data factory data_ is using local data access: input_folder - output/04_fuzzy_dedupe_out/cleaned output_folder - output/05_doc_quality_out
00:15:28 INFO - data factory data_ max_files -1, n_sample -1
00:15:28 INFO - data fac

✅ Stage:5 completed successfully
CPU times: user 38.8 ms, sys: 2.53 ms, total: 41.3 ms
Wall time: 35.3 ms


### 8.2 - Inspect the Output

We will see several new columns starting with the name **docq_**.

We will look at a metric **docq_contain_bad_word** and filter out any documents that have bad words.

For more information see : [Doc Quality documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/doc_quality)

In [19]:
output_df = read_parquet_files_as_df(output_doc_quality_dir)
print ("Displaying contents of : ", output_doc_quality_dir)
output_df.head()

Displaying contents of :  output/05_doc_quality_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
0,spam.pdf,Free xxx,1,0,2,5a9d562d-ba87-4b2f-954d-fd9e7aece509,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,...,3.500000,0.000000,1,0.000000,0.0,True,0.000000,0.0,1.000000,False
1,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,...,4.688000,0.032000,8,0.000000,0.0,False,0.176471,0.0,0.880000,True
2,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,...,4.541284,0.027523,9,0.000000,0.0,False,0.176471,0.0,0.880734,True
3,lorem.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,8a86db6e-0ac6-480c-9a86-d4ac383e2589,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,...,5.000000,0.000000,1,0.085714,0.0,False,0.000000,0.0,1.000000,False


### 8.3 - Filtering 'quality' documents

So from the output above we see **spam.pdf** is flagged for containing bad words (see column **docq_contain_bad_word**).

Also **lorem.pdf** is flagged for place holder content **lorem ipsum**

We are going to filter them both out

In [22]:
all_docs_df = read_parquet_files_as_df(output_doc_quality_dir)

# remove documents with badwords
clean_docs = all_docs_df[all_docs_df['docq_contain_bad_word'] == False]

# also filter out 'lorem ipsum' text
clean_docs = clean_docs[clean_docs['docq_lorem_ipsum_ratio'] == 0]

clean_docs.head()

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
1,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,0dd165a1-6de2-4df0-ad7a-c6ad21da9c18,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,...,4.688000,0.032000,8,0.0,0.0,False,0.176471,0.0,0.880000,True
2,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,27d9fd2f-d815-4937-bbbf-d4a10cbce4c6,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,...,4.541284,0.027523,9,0.0,0.0,False,0.176471,0.0,0.880734,True


## Step-9: Copy output to final output dir

In [21]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER_FINAL, exist_ok=True)

clean_docs.to_parquet(os.path.join(MY_CONFIG.OUTPUT_FOLDER_FINAL, "final.parquet"))
print (f"✅ Saved output to '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Saved output to 'output/output_final'
